In [7]:
from boto.s3.connection import S3Connection
import boto3

In [8]:
s3_client = boto3.client("s3")

In [9]:
### CRAWL FILE-SYSTEM ###
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

knowledge_path = "/Users/joshua.sheppard/full_wiki_extract"

def iter_filesys(path):
    if os.path.isfile(path):
        yield path

    elif os.path.isdir(path):
        for dir, _, filenames in os.walk(path):
                for f in filenames:
                    if not f.endswith('.DS_Store'):
                        yield os.path.join(dir, f)

    else:
        raise RuntimeError("Invalid path %s" % path)

kw_files = iter_filesys(knowledge_path)

kw_sample = []
for i in kw_files:
    kw_sample.append(i)

print(kw_sample)
print(len(kw_sample))

['/Users/joshua.sheppard/full_wiki_extract/GA/wiki_73', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_87', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_80', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_74', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_89', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_42', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_45', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_11', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_16', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_29', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_20', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_27', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_18', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_44', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_88', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_43', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_75', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_81', '/Users/j

In [13]:
import os
bucket = "wiki-evidence"
worklist = kw_sample
batchsize = 500

for i in range(0, len(worklist), batchsize):
    print(f".... [Batch {i}].....")
    batch = worklist[i: i + batchsize]

    for j in range(0, len(batch)):
        s3_client.upload_file(kw_sample[j], bucket, os.path.basename(os.path.normpath(kw_sample[j])))

.... [Batch 0].....
.... [Batch 500].....


KeyboardInterrupt: 

In [ ]:
### CONSTRUCT SQLITE DB ###

# TODOs: Implement Multiprocessing



In [ ]:
import sqlite3
from multiprocessing import Pool as ProcessPool


def store_contents(data_path, save_path, preprocess, num_workers=None):
    """Preprocess and store a corpus of documents in sqlite.

    Args:
        data_path: Root path to directory (or directory of directories) of files
          containing json encoded documents (must have `id` and `text` fields).
        save_path: Path to output sqlite db.
        preprocess: Path to file defining a custom `preprocess` function. Takes
          in and outputs a structured doc.
        num_workers: Number of parallel processes to use when reading docs.
    """
    if os.path.isfile(save_path):
        raise RuntimeError('%s already exists! Not overwriting.' % save_path)

    logger.info('Reading into database...')
    conn = sqlite3.connect(save_path)
    c = conn.cursor()
    c.execute("CREATE TABLE documents (id PRIMARY KEY, text);")

    workers = ProcessPool(num_workers, initializer=init, initargs=(preprocess,))
    files = [f for f in kw_files]
    count = 0
    with tqdm(total=len(files)) as pbar:
        for pairs in tqdm(workers.imap_unordered(get_contents, files)):
            count += len(pairs)
            c.executemany("INSERT INTO documents VALUES (?,?)", pairs)
            pbar.update()
    logger.info('Read %d docs.' % count)
    logger.info('Committing...')
    conn.commit()
    conn.close()

In [ ]:
import spacy
import tqdm

nlp = spacy.load("en_core_web_sm")

#workers = ProcessPool(3)
def paragraphs(document):
    start = 0
    document = nlp(document)
    passages = []
    for token in document:
        if token.is_space and token.text.count("\n") > 1:
            yield document[start:token.i]
            start = token.i
    yield document[start:]

files = [f for f in kw_sample]
count = 0
with tqdm(total=len(files)) as pbar:
    for i in files:
        ev = get_contents(i)
        for j in ev:
            _id, title, text = j
            passages = [str(i) for i in paragraphs(text)][0].split("\n")
            for passage in passages:
                if len(passage) < 50:
                    continue

                count += 1
                doc = {"id": _id, "title": title, "text": passage}
                wiki_ev.add_doc(doc)

    pbar.update()

In [ ]:
logger.info('Reading into database...')

# conn = sqlite3.connect(save_path)
# c = conn.cursor()
# c.execute("CREATE TABLE documents (id PRIMARY KEY, text);")

wiki_ev.add_doc(doc)

workers = ProcessPool(2)
files = [f for f in kw_files]
count = 0
with tqdm(total=len(files)) as pbar:
    for pairs in tqdm(workers.imap_unordered(get_contents, files)):
        count += len(pairs)
        c.executemany("INSERT INTO documents VALUES (?,?)", pairs)
        pbar.update()
logger.info('Read %d docs.' % count)
logger.info('Committing...')
conn.commit()
conn.close()